In [2]:
import os

if not os.getcwd().endswith('mob2crime'):
    os.chdir('..')
os.getcwd()

'/home/Jiahui/mob2crime'

In [36]:
import geopandas as gp
from shapely.geometry import Polygon, LineString, box
import numpy as np
import fiona

In [4]:
%matplotlib inline

In [5]:
%load_ext line_profiler

In [6]:
poly = Polygon([[-99.2724609375,19.539731238254422],[-99.2779541015625,19.452348936859018],[-99.25529479980469,19.344188652729514],[-99.1461181640625,19.36751071819971],[-99.13444519042967,19.497664168139053],[-99.20516967773438,19.561731187837697],[-99.1413116455078,19.583728136218753],[-99.24842834472656,19.594725484073244],[-99.27108764648438,19.54943746814108],[-99.28138732910156,19.54296671307226],[-99.2724609375,19.539731238254422]])

In [7]:
def poly_bbox(poly):
    # poly can be a 4-element tuple representing the bounding box: lon_min, lat_min, lon_max, lat_max
    # or a closed LineString representing the outer ring
    if isinstance(poly, tuple):
        if len(poly) == 4:
            lon_min, lat_min, lon_max, lat_max = poly
            poly = box(poly)
        else:
            raise ValueError('poly is a tuple, but its len != 4')
    elif isinstance(poly, LineString):
        if poly.is_closed:
            lon_min, lat_min, lon_max, lat_max = poly.bounds
            poly = Polygon(poly)
        else:
            raise ValueError('poly is LineString but not closed, which is not supported here')
    elif isinstance(poly, Polygon):
        lon_min, lat_min, lon_max, lat_max = poly.bounds
    else:
        raise ValueError('poly is not bbox tuple, closed LineString or Polygon')
    return poly, lon_min, lat_min, lon_max, lat_max

In [8]:
def poly2grids(poly, side, within_poly=True):
    # poly can be a 4-element tuple representing the bounding box: lon_min, lat_min, lon_max, lat_max
    # or a closed LineString representing the outer ring

    poly, lon_min, lat_min, lon_max, lat_max = poly_bbox(poly)

    grids_lon, grids_lat = np.mgrid[lon_min:(lon_max + side):side, lat_min:(lat_max + side):side]
    nlon,nlat = grids_lon.shape

    grids = []
    for i in range(nlon-1):
        for j in range(nlat-1):
            g = box(grids_lon[i, j], grids_lat[i, j], grids_lon[i + 1, j + 1], grids_lat[i + 1, j + 1])
            if not g.intersects(poly):
                continue
            if within_poly and not g.within(poly): 
                g = g.intersection(poly)
            grids.append(g)
    return grids

In [8]:
%lprun -f poly2grids poly2grids(poly, 0.001, False)

In [9]:
gp_poly = gp.GeoDataFrame([poly], columns=['geometry'])

In [10]:
gp_poly.crs = {'init': 'epsg:4326'}

In [67]:
def gp_polys_to_grids(gp_polys, side, cur_crs=None, eqdc_crs=None):
    """if gpdf has crs, use its own; else use cur_crs
    """
    cur_crs = crs_normalization(cur_crs)
    eqdc_crs = crs_normalization(eqdc_crs)
    
    if eqdc_crs:  # crs transform is needed
        assign_crs(gp_polys, cur_crs)
        cur_crs = gp_polys.crs  # store the original crs
        gp_polys = gp_polys.to_crs(eqdc_crs)
    
    indices = []
    grids = []
    for i, row in gp_polys.iterrows():
        gs = poly2grids(row.geometry, side)
        grids.extend(gs)
        indices.extend([i]*len(gs))
    grids = gp.GeoDataFrame(list(zip(indices, grids)), columns=['poly', 'geometry'])
    grids.crs = eqdc_crs
    if eqdc_crs:
        grids = grids.to_crs(cur_crs)
    return grids

In [55]:
eqdc = '+proj=eqdc +lat_0=40 +lon_0=-96 +lat_1=20 +lat_2=60 +x_0=0 +y_0=0 +datum=NAD83 +units=m +no_defs'

In [70]:
%lprun -f gp_polys_to_grids gp_polys_to_grids(gp_poly,1000, eqdc_crs=eqdc)

In [68]:
grids = gp_polys_to_grids(gp_poly,1000, eqdc_crs=eqdc)

grids.crs, gp_poly.crs

({'init': 'epsg:4326'}, {'init': 'epsg:4326'})

In [71]:
grids.head()

,poly,geometry
0,0,"POLYGON ((-99.266569385471 19.39804056884898, ..."
1,0,POLYGON ((-99.26844984705234 19.40701572776243...
2,0,POLYGON ((-99.27033069865851 19.41599084079635...
3,0,POLYGON ((-99.27221194040925 19.42496590793591...
4,0,POLYGON ((-99.27409357242436 19.43394092916626...
